# Evaluate the Trained LLM

## Import the needed Library

In [1]:

%pip install openai pytest pandas python-dotenv scikit-learn tiktoken matplotlib nltk spacy rouge-score bert-score numpy


import openai
import pandas as pd
import pytest
import json
from dotenv import load_dotenv
import os
from sklearn.model_selection import train_test_split
import base64
import re
import tiktoken
import matplotlib.pyplot as plt

from nltk.translate.bleu_score import sentence_bleu
import spacy
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from bert_score import score
import numpy as np
from transformers import BertTokenizer, BertModel


Note: you may need to restart the kernel to use updated packages.



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Users/aaronz/anaconda3/envs/cura/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/aaronz/anaconda3/envs/cura/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/Users/aaronz/anaconda3/envs/cura/lib/python3.9/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/aaronz/anaconda3/envs/cura/lib/python3.9/site-packages/traitlets/config/application.py", line 1075, in l

In [2]:
df_test = pd.read_csv('data/generated/generated_answers.csv')
df_train = pd.read_csv('data/generated/generated_train_answers.csv')
df_validation = pd.read_csv('data/generated/generated_validate_answers.csv')

In [3]:
df_test

,official_answer,fine_tunned_answer,4o_mini_answer
0,"""The liquid reduced because it evaporated. It ...","""The liquid in the pan reduced as it evaporate...",The liquid in the pan reduced due to the proce...
1,"""Student finger, rock or stone, wood, thick fa...","""Aluminum foil and plastic wrap. We could feel...",Exemplar Answer:\n\nThe opaque objects that ab...
2,"""The ova (eggs) or ovules are protected from d...","""It is important that the ovules are found in ...",The ovules are located within the ovary of the...
3,"""My conclusions would have been different, as ...","""It would have been that ostracism was a fair ...","If I had only examined Source 1, my conclusion..."
4,"""This experiment uses milk which contains the ...","""Trypsin breaks down proteins into smaller mol...","In the trypsin experiment, the results indicat..."
5,"""Tension force. Tension force = weight force. ...","""Tension force. Tension force = weight of mass...",The force present in the string when tied to t...
6,"""Fin designs that have three or four triangula...","""The fin design with the most surface area wor...",The fin design that worked best was the triang...
7,"""Oobleck acts like a solid if it has very litt...","""Oobleck behaves like a liquid when it is left...",Oobleck exhibits properties of both a liquid a...
8,"""The best results were when we used two or thr...","""Using two mirrors gave the brightest light sp...",The method that gave the brightest light spot ...
9,"""Mars, Earth__ moon, Europa, Enceladus, and Ti...","""Earth's moon, Mars, Europa, Enceladus and Tit...",The most likely places for us to live on in ou...


In [4]:
df_train

,official_answer,fine_tunned_answer,4o_mini_answer
0,"""The correct answer to this question will depe...","""The filter paper was able to separate out all...",The filter paper was not able to separate out ...
1,"""Although it seems democratic, ostracism was n...","""Ostracism was a democratic process that allow...",Ostracism in Ancient Greece was a democratic p...
2,"""We left the lid off Jar 1 so that air (oxygen...","""We left the lid off Jar 1 so that it could ha...",We left the lid off Jar 1 to allow oxygen from...
3,"""Group A - Regional/rural townGroup B - Smalle...","""Group 1 - Large cityGroup 2 - Smaller cityGro...","In the context of the provided information, th..."
4,"""They do not tell us how exactly the voting pr...","""These sources do not tell us how many votes w...",The sources do not provide insight into the so...
5,"""Please note that the above graph is an illust...","""Students should create a line graph that show...",**Exemplar Answer:**\n\n**Title:** Change in S...
6,"""Yes, it could as you could try to pump oil-po...","""Yes - if there was a small oil spill, this me...","Yes, this method of separating oil and water c..."
7,"""The first four planets are relatively close t...","""The first four planets are relatively close t...",The distances of the first four planets - Merc...
8,"""The ratio of the load distance to effort dist...","""The ratio of the load distance to effort dist...",To determine the ratio of the load distance to...
9,"""Stormwater which runs through a sediment pond...","""Stormwater which flows through a sediment bas...",Stormwater that flows through a sediment basin...


In [5]:
df_validation

,official_answer,fine_tunned_answer,4o_mini_answer
0,"""The Acropolis has been subject to a lot of da...","""The Acropolis has faced many threats over the...",The Acropolis faces several past and current t...
1,"""I think the Colosseum should be preserved as ...","""It is great that this site has been preserved...",One historical site I would like to visit is t...
2,"""Savannas. Average precipitation ranges betwee...","""Temperate grasslands have the greatest variat...",The biome with the greatest variation in annua...
3,"""Wax paper, tissue paper, notebook paper, and ...","""Wax paper and some plastics. We could see tha...",Exemplar Answer:\n\nThe materials that were tr...
4,"""Yes, it does matter. Each species of bee is a...","""It does matter if one or two species of bees ...",It does matter if one or two species of bees b...
5,"""Shortest at or close to midday. Longest early...","""The shadow is at its shortest at midday. This...","The shadow is at its shortest around noon, whe..."
6,"""The Earth’s orbit around the Sun is an ellipt...","""The Earth’s orbit is elliptical (oval) in shape""",The shape of the Earth's orbit around the Sun ...
7,"""An oil spill on a lake would be easier to cle...","""An oil spill on a lake would be easier to cle...",An oil spill on a lake is generally easier to ...
8,"""The ratio of the load force to the effort for...","""The ratio of the load force to effort force i...",To determine the ratio of the load force to th...
9,"""Tea, flour and cooking oil. These substances ...","""Flour and cooking oil. They did not dissolve ...","In this experiment, the substances that were i..."


## Preprocessing the generated data

In [6]:
nlp = spacy.load("en_core_web_sm")

contractions = {
    "can't": "cannot",
    "won't": "will not",
    "i'm": "I am",
    "isn't": "is not",
    "didn't": "did not",
    "couldn't": "could not",
    "doesn't": "does not",
    "it's": "it is",
    "let's": "let us",
    "they're": "they are"
}

def preprocess_text_with_spacy(text):
    # Contractions
    for contraction, full_form in contractions.items():
        text = text.replace(contraction, full_form)

    doc = nlp(text)

    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]

    cleaned_text = " ".join(tokens).lower().strip()

    return cleaned_text



df_test['official_answer'] = df_train['official_answer'].apply(preprocess_text_with_spacy)
df_test['fine_tunned_answer'] = df_train['fine_tunned_answer'].apply(preprocess_text_with_spacy)
df_test['4o_mini_answer'] = df_train['4o_mini_answer'].apply(preprocess_text_with_spacy)

df_train['official_answer'] = df_train['official_answer'].apply(preprocess_text_with_spacy)
df_train['fine_tunned_answer'] = df_train['fine_tunned_answer'].apply(preprocess_text_with_spacy)
df_train['4o_mini_answer'] = df_train['4o_mini_answer'].apply(preprocess_text_with_spacy)

df_validation['official_answer'] = df_validation['official_answer'].apply(preprocess_text_with_spacy)
df_validation['fine_tunned_answer'] = df_validation['fine_tunned_answer'].apply(preprocess_text_with_spacy)
df_validation['4o_mini_answer'] = df_validation['4o_mini_answer'].apply(preprocess_text_with_spacy)


In [7]:
df_validation

,official_answer,fine_tunned_answer,4o_mini_answer
0,acropolis subject lot damage year cause war pa...,acropolis face threat year past damage war ear...,acropolis face past current threat jeopardize ...
1,think colosseum preserve connect people past h...,great site preserve connect past help understa...,historical site like visit pyramid giza egypt ...
2,savannas average precipitation range 50 275 cm...,temperate grassland great variation annual pre...,biome great variation annual precipitation tro...
3,wax paper tissue paper notebook paper vegetabl...,wax paper plastic light transmit,exemplar answer \n\n material translucent expe...
4,yes matter specie bee adapt fulfil particular ...,matter specie bee extinct different specie bee...,matter specie bee extinct specie play unique r...
5,shortest close midday long early morning late ...,shadow short midday sun high point sky time sh...,shadow short noon sun high point sky time sun ...
6,earth orbit sun elliptical oval shape mean ear...,earth orbit elliptical oval shape,shape earth orbit sun elliptical mean resemble...
7,oil spill lake easy clean oil stay layer easy ...,oil spill lake easy clean ocean water clean fe...,oil spill lake generally easy clean ocean reas...
8,ratio load force effort force 2:1,ratio load force effort force experiment 2:1 r...,determine ratio load force effort force experi...
9,tea flour cooking oil substance dissolve disap...,flour cooking oil dissolve water float water,experiment substance insoluble water flour coo...


## Evaluate

### BLEU (Bilingual Evaluation Understudy)

In [8]:
df_test['BLEU_fine_tune'] = None
df_test['BLEU_o4_mini'] = None

df_train['BLEU_fine_tune'] = None
df_train['BLEU_o4_mini'] = None

df_validation['BLEU_fine_tune'] = None
df_validation['BLEU_o4_mini'] = None

In [9]:
smoothie = SmoothingFunction().method1
def calculate_bleu_score(df):
  
    for i in range(len(df)):
        print(f"Start calculate the BLEU score for data: line{i}")

        standard = [df.iloc[i]["official_answer"].split()]
        fine_tunned_answer = df.iloc[i]["fine_tunned_answer"].split()
        mini_answer = df.iloc[i]["4o_mini_answer"].split()

        bleu_score_fine_tune = sentence_bleu(standard, fine_tunned_answer,smoothing_function=smoothie)
        bleu_score_mini = sentence_bleu(standard, mini_answer,smoothing_function=smoothie)

        df.loc[i,"BLEU_fine_tune"] = bleu_score_fine_tune
        df.loc[i,"BLEU_o4_mini"] = bleu_score_mini
    
    return df


In [10]:
df_test = calculate_bleu_score(df_test)
df_train = calculate_bleu_score(df_train)
df_validation = calculate_bleu_score(df_validation)

Start calculate the BLEU score for data: line0
Start calculate the BLEU score for data: line1
Start calculate the BLEU score for data: line2
Start calculate the BLEU score for data: line3
Start calculate the BLEU score for data: line4
Start calculate the BLEU score for data: line5
Start calculate the BLEU score for data: line6
Start calculate the BLEU score for data: line7
Start calculate the BLEU score for data: line8
Start calculate the BLEU score for data: line9
Start calculate the BLEU score for data: line10
Start calculate the BLEU score for data: line11
Start calculate the BLEU score for data: line12
Start calculate the BLEU score for data: line13
Start calculate the BLEU score for data: line14
Start calculate the BLEU score for data: line15
Start calculate the BLEU score for data: line16
Start calculate the BLEU score for data: line17
Start calculate the BLEU score for data: line0
Start calculate the BLEU score for data: line1
Start calculate the BLEU score for data: line2
Start

In [11]:
df_validation

,official_answer,fine_tunned_answer,4o_mini_answer,BLEU_fine_tune,BLEU_o4_mini
0,acropolis subject lot damage year cause war pa...,acropolis face threat year past damage war ear...,acropolis face past current threat jeopardize ...,0.00772,0.005465
1,think colosseum preserve connect people past h...,great site preserve connect past help understa...,historical site like visit pyramid giza egypt ...,0.02245,0.002826
2,savannas average precipitation range 50 275 cm...,temperate grassland great variation annual pre...,biome great variation annual precipitation tro...,0.012301,0.00302
3,wax paper tissue paper notebook paper vegetabl...,wax paper plastic light transmit,exemplar answer \n\n material translucent expe...,0.017018,0.010608
4,yes matter specie bee adapt fulfil particular ...,matter specie bee extinct different specie bee...,matter specie bee extinct specie play unique r...,0.046399,0.021859
5,shortest close midday long early morning late ...,shadow short midday sun high point sky time sh...,shadow short noon sun high point sky time sun ...,0.108401,0.099555
6,earth orbit sun elliptical oval shape mean ear...,earth orbit elliptical oval shape,shape earth orbit sun elliptical mean resemble...,0.10071,0.052033
7,oil spill lake easy clean oil stay layer easy ...,oil spill lake easy clean ocean water clean fe...,oil spill lake generally easy clean ocean reas...,0.237751,0.016037
8,ratio load force effort force 2:1,ratio load force effort force experiment 2:1 r...,determine ratio load force effort force experi...,0.331808,0.034131
9,tea flour cooking oil substance dissolve disap...,flour cooking oil dissolve water float water,experiment substance insoluble water flour coo...,0.046454,0.010052


### ROUGE

In [12]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
def calculate_rouge_score(df):
    
    P_rogue1_fine = []
    R_rogue1_fine = []
    F1_rogue1_fine = []

    P_rougeL_fine = []
    R_rougeL_fine = []
    F1_rougeL_fine = []

    P_rogue1_mini = []
    R_rogue1_mini = []
    F1_rogue1_mini = []

    P_rougeL_mini = []
    R_rougeL_mini = []
    F1_rougeL_mini = []
    
    for i in range(len(df)):
            print(f"Start calculate the ROUGE score for data: line{i}")

            standard = df.iloc[i]["official_answer"]
            fine_tunned_answer = df.iloc[i]["fine_tunned_answer"]
            mini_answer = df.iloc[i]["4o_mini_answer"]

            rouge_score_fine_tune = scorer.score(standard, fine_tunned_answer)
            rouge_score_mini = scorer.score(standard, mini_answer)

            P_rogue1_fine.append(rouge_score_fine_tune['rouge1'].precision)
            R_rogue1_fine.append(rouge_score_fine_tune['rouge1'].recall)
            F1_rogue1_fine.append(rouge_score_fine_tune['rouge1'].fmeasure)

            P_rougeL_fine.append(rouge_score_fine_tune['rougeL'].precision)
            R_rougeL_fine.append(rouge_score_fine_tune['rougeL'].recall)
            F1_rougeL_fine.append(rouge_score_fine_tune['rougeL'].fmeasure)

            P_rogue1_mini.append(rouge_score_mini['rouge1'].precision)
            R_rogue1_mini.append(rouge_score_mini['rouge1'].recall)
            F1_rogue1_mini.append(rouge_score_mini['rouge1'].fmeasure)

            P_rougeL_mini.append(rouge_score_mini['rouge1'].precision)
            R_rougeL_mini.append(rouge_score_mini['rouge1'].recall)
            F1_rougeL_mini.append(rouge_score_mini['rouge1'].fmeasure)
            
    this_df = pd.DataFrame({
        'P_rogue1_fine': P_rogue1_fine,
        'R_rogue1_fine': R_rogue1_fine,
        'F1_rogue1_fine': F1_rogue1_fine,
        'P_rougeL_fine': P_rougeL_fine,
        'R_rougeL_fine': R_rougeL_fine,
        'F1_rougeL_fine': F1_rougeL_fine,
        'P_rogue1_mini': P_rogue1_mini,
        'R_rogue1_mini': R_rogue1_mini,
        'F1_rogue1_mini': F1_rogue1_mini,
        'P_rougeL_mini': P_rougeL_mini,
        'R_rougeL_mini': R_rougeL_mini,
        'F1_rougeL_mini': F1_rougeL_mini
    })

    return this_df

In [13]:
df_test_rouge = calculate_rouge_score(df_test)
df_train_rouge = calculate_rouge_score(df_train)
df_validation_rouge = calculate_rouge_score(df_validation)

Start calculate the ROUGE score for data: line0
Start calculate the ROUGE score for data: line1
Start calculate the ROUGE score for data: line2
Start calculate the ROUGE score for data: line3
Start calculate the ROUGE score for data: line4
Start calculate the ROUGE score for data: line5
Start calculate the ROUGE score for data: line6
Start calculate the ROUGE score for data: line7
Start calculate the ROUGE score for data: line8
Start calculate the ROUGE score for data: line9
Start calculate the ROUGE score for data: line10
Start calculate the ROUGE score for data: line11
Start calculate the ROUGE score for data: line12
Start calculate the ROUGE score for data: line13
Start calculate the ROUGE score for data: line14
Start calculate the ROUGE score for data: line15
Start calculate the ROUGE score for data: line16
Start calculate the ROUGE score for data: line17
Start calculate the ROUGE score for data: line0
Start calculate the ROUGE score for data: line1
Start calculate the ROUGE score 

### BERTScore

In [14]:



print(f"Start calculate the Bert score for data")

standard = df_test["official_answer"].tolist()
fine_tunned_answer = df_test["fine_tunned_answer"].tolist()
mini_answer = df_test["4o_mini_answer"].tolist()

df_test_bert_score_fine_tune = score(fine_tunned_answer, standard, lang="en", rescale_with_baseline=False)
df_test_bert_score_mini = score(mini_answer, standard, lang="en", rescale_with_baseline=False)


standard = df_train["official_answer"].tolist()
fine_tunned_answer = df_train["fine_tunned_answer"].tolist()
mini_answer = df_train["4o_mini_answer"].tolist()

df_train_bert_score_fine_tune = score(fine_tunned_answer, standard, lang="en", rescale_with_baseline=False)
df_train_bert_score_mini = score(mini_answer, standard, lang="en", rescale_with_baseline=False)


standard = df_validation["official_answer"].tolist()
fine_tunned_answer = df_validation["fine_tunned_answer"].tolist()
mini_answer = df_validation["4o_mini_answer"].tolist()

df_validation_bert_score_fine_tune = score(fine_tunned_answer, standard, lang="en", rescale_with_baseline=False)
df_validation_bert_score_mini = score(mini_answer, standard, lang="en", rescale_with_baseline=False)


Start calculate the Bert score for data


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['ro

In [15]:
# BERT Score ：P（precision）, R（recall）, F1
P_test, R_test, F1_test = df_test_bert_score_fine_tune
P_train, R_train, F1_train = df_train_bert_score_fine_tune
P_validation, R_validation, F1_validation = df_validation_bert_score_fine_tune

P_test_o, R_test_o, F1_test_o = df_test_bert_score_mini
P_train_o, R_train_o, F1_train_o = df_train_bert_score_mini
P_validation_o, R_validation_o, F1_validation_o = df_validation_bert_score_mini


bert_data = {
    "P_train": P_train.tolist(),
    "P_train_o": P_train_o.tolist(),
    "P_test": P_test.tolist(),
    "P_test_o": P_test_o.tolist(),
    "P_validation": P_validation.tolist(),
    "P_validation_o": P_validation_o.tolist(),
    "R_train": R_train.tolist(),
    "R_train_o": R_train_o.tolist(),
    "R_test": R_test.tolist(),
    "R_test_o": R_test_o.tolist(),
    "R_validation": R_validation.tolist(),
    "R_validation_o": R_validation_o.tolist(),
    "F1_train": F1_train.tolist(),
    "F1_train_o": F1_train_o.tolist(),
    "F1_test": F1_test.tolist(),
    "F1_test_o": F1_test_o.tolist(),
    "F1_validation": F1_validation.tolist(),
    "F1_validation_o": F1_validation_o.tolist(),
}


bert_score_df = pd.DataFrame(bert_data)
bert_score_df

,P_train,P_train_o,P_test,P_test_o,P_validation,P_validation_o,R_train,R_train_o,R_test,R_test_o,R_validation,R_validation_o,F1_train,F1_train_o,F1_test,F1_test_o,F1_validation,F1_validation_o
0,0.851073,0.830172,0.851073,0.830172,0.862566,0.823095,0.820701,0.841389,0.820701,0.841389,0.844664,0.851107,0.835611,0.835743,0.835611,0.835743,0.853521,0.836866
1,0.837143,0.802147,0.837143,0.802147,0.863262,0.793842,0.812836,0.828479,0.812836,0.828479,0.831809,0.842845,0.824810,0.815100,0.824810,0.815100,0.847244,0.817610
2,0.938886,0.858702,0.938886,0.858702,0.846628,0.793292,0.895425,0.903526,0.895425,0.903526,0.823736,0.819273,0.916641,0.880544,0.916641,0.880544,0.835025,0.806073
3,0.915542,0.780460,0.915542,0.780460,0.901733,0.803790,0.887759,0.828171,0.887759,0.828171,0.840561,0.851488,0.901437,0.803608,0.901437,0.803608,0.870074,0.826952
4,0.820946,0.800748,0.820946,0.800748,0.870939,0.833948,0.808682,0.816048,0.808682,0.816048,0.856189,0.866599,0.814768,0.808326,0.814768,0.808326,0.863501,0.849960
5,0.784017,0.726955,0.784017,0.726955,0.854362,0.794522,0.831832,0.825357,0.831832,0.825357,0.852210,0.831202,0.807217,0.773037,0.807217,0.773037,0.853284,0.812448
6,0.837947,0.815765,0.837947,0.815765,0.956940,0.834840,0.819633,0.854859,0.819633,0.854859,0.883789,0.898623,0.828689,0.834854,0.828689,0.834854,0.918911,0.865558
7,0.940437,0.809727,0.940437,0.809727,0.878161,0.807265,0.811009,0.859867,0.811009,0.859867,0.862651,0.880710,0.870941,0.834044,0.870941,0.834044,0.870336,0.842390
8,1.000000,0.782535,1.000000,0.782535,0.909359,0.782460,1.000000,0.853342,1.000000,0.853342,0.950481,0.886862,1.000000,0.816406,1.000000,0.816406,0.929466,0.831397
9,0.862328,0.844507,0.862328,0.844507,0.899151,0.818273,0.846845,0.856490,0.846845,0.856490,0.858068,0.870399,0.854517,0.850456,0.854517,0.850456,0.878129,0.843531


## Draw out the statistical table

### BLEU (Bilingual Evaluation Understudy)

In [16]:
bleu_data_fine_tune = {
    "BLEU_train": df_train["BLEU_fine_tune"].tolist(),
    "BLEU_test": df_test["BLEU_fine_tune"].tolist(),
    "BLEU_validation": df_validation["BLEU_fine_tune"].tolist(),
}

bleu_data_df_fine_tune = pd.DataFrame(bleu_data_fine_tune)
bleu_data_df_fine_tune

,BLEU_train,BLEU_test,BLEU_validation
0,0.017320,0.017320,0.007720
1,0.014133,0.014133,0.022450
2,0.159766,0.159766,0.012301
3,0.037205,0.037205,0.017018
4,0.012992,0.012992,0.046399
5,0.005311,0.005311,0.108401
6,0.015992,0.015992,0.100710
7,0.142762,0.142762,0.237751
8,1.000000,1.000000,0.331808
9,0.026998,0.026998,0.046454


In [17]:
bleu_data_4o= {
    "BLEU_train": df_train["BLEU_o4_mini"].tolist(),
    "BLEU_test": df_test["BLEU_o4_mini"].tolist(),
    "BLEU_validation": df_validation["BLEU_o4_mini"].tolist(),
}

bleu_data_df_4o = pd.DataFrame(bleu_data_4o)
bleu_data_df_4o

,BLEU_train,BLEU_test,BLEU_validation
0,0.069593,0.069593,0.005465
1,0.003716,0.003716,0.002826
2,0.075166,0.075166,0.003020
3,0.005170,0.005170,0.010608
4,0.000000,0.000000,0.021859
5,0.001843,0.001843,0.099555
6,0.006000,0.006000,0.052033
7,0.011917,0.011917,0.016037
8,0.029685,0.029685,0.034131
9,0.031239,0.031239,0.010052


In [18]:
bleu_data_df_fine_tune.to_csv('data/statistic/bleu_data_fine_tune.csv', index=False)
bleu_data_df_4o.to_csv('data/statistic/bleu_data_4o.csv', index=False)

In [19]:
bleu_data_df_fine_tune.describe()

,BLEU_train,BLEU_test,BLEU_validation
count,18.000000,18.000000,18.000000
mean,0.207368,0.207368,0.073503
std,0.370137,0.370137,0.084942
min,0.000000,0.000000,0.000027
25%,0.013943,0.013943,0.018659
50%,0.022812,0.022812,0.048081
75%,0.155515,0.155515,0.093179
max,1.000000,1.000000,0.331808


In [20]:
bleu_data_df_4o.describe()

,BLEU_train,BLEU_test,BLEU_validation
count,18.000000,18.000000,18.000000
mean,0.022617,0.022617,0.022767
std,0.024128,0.024128,0.026662
min,0.000000,0.000000,0.002826
25%,0.004079,0.004079,0.005691
50%,0.015853,0.015853,0.013050
75%,0.029567,0.029567,0.024225
max,0.075166,0.075166,0.099555


### ROUGE

In [21]:
df_test_rouge.to_csv('data/statistic/df_test_rouge.csv', index=False)
df_train_rouge.to_csv('data/statistic/df_train_rouge.csv', index=False)
df_validation_rouge.to_csv('data/statistic/df_validation_rouge.csv', index=False)

In [22]:
df_test_rouge_desc = df_test_rouge.describe()
df_train_rouge_desc = df_train_rouge.describe()
df_validation_rouge_desc = df_validation_rouge.describe()

In [23]:
df_test_rouge_desc

,P_rogue1_fine,R_rogue1_fine,F1_rogue1_fine,P_rougeL_fine,R_rougeL_fine,F1_rougeL_fine,P_rogue1_mini,R_rogue1_mini,F1_rogue1_mini,P_rougeL_mini,R_rougeL_mini,F1_rougeL_mini
count,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000
mean,0.566660,0.405471,0.458744,0.505254,0.365308,0.410827,0.168641,0.517315,0.236744,0.168641,0.517315,0.236744
std,0.340933,0.313005,0.313879,0.336888,0.315584,0.314585,0.127717,0.270135,0.155284,0.127717,0.270135,0.155284
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.312112,0.211466,0.253247,0.291502,0.185924,0.227381,0.071992,0.422932,0.123368,0.071992,0.422932,0.123368
50%,0.583591,0.316958,0.422035,0.424812,0.240385,0.321662,0.147356,0.482265,0.246964,0.147356,0.482265,0.246964
75%,0.878125,0.527059,0.616635,0.737500,0.404977,0.546296,0.289448,0.743077,0.393407,0.289448,0.743077,0.393407
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.385542,1.000000,0.444444,0.385542,1.000000,0.444444


In [24]:
df_train_rouge_desc

,P_rogue1_fine,R_rogue1_fine,F1_rogue1_fine,P_rougeL_fine,R_rougeL_fine,F1_rougeL_fine,P_rogue1_mini,R_rogue1_mini,F1_rogue1_mini,P_rougeL_mini,R_rougeL_mini,F1_rougeL_mini
count,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000
mean,0.566660,0.405471,0.458744,0.505254,0.365308,0.410827,0.168641,0.517315,0.236744,0.168641,0.517315,0.236744
std,0.340933,0.313005,0.313879,0.336888,0.315584,0.314585,0.127717,0.270135,0.155284,0.127717,0.270135,0.155284
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.312112,0.211466,0.253247,0.291502,0.185924,0.227381,0.071992,0.422932,0.123368,0.071992,0.422932,0.123368
50%,0.583591,0.316958,0.422035,0.424812,0.240385,0.321662,0.147356,0.482265,0.246964,0.147356,0.482265,0.246964
75%,0.878125,0.527059,0.616635,0.737500,0.404977,0.546296,0.289448,0.743077,0.393407,0.289448,0.743077,0.393407
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.385542,1.000000,0.444444,0.385542,1.000000,0.444444


In [25]:
df_validation_rouge_desc

,P_rogue1_fine,R_rogue1_fine,F1_rogue1_fine,P_rougeL_fine,R_rougeL_fine,F1_rougeL_fine,P_rogue1_mini,R_rogue1_mini,F1_rogue1_mini,P_rougeL_mini,R_rougeL_mini,F1_rougeL_mini
count,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000
mean,0.532621,0.363107,0.401646,0.486274,0.322350,0.359636,0.159907,0.585096,0.240264,0.159907,0.585096,0.240264
std,0.236176,0.189044,0.156504,0.261236,0.202025,0.174610,0.077530,0.189736,0.097978,0.077530,0.189736,0.097978
min,0.125000,0.051282,0.093023,0.062500,0.051282,0.080000,0.050633,0.153846,0.090909,0.050633,0.153846,0.090909
25%,0.415966,0.272727,0.308333,0.293233,0.186364,0.241667,0.106271,0.475000,0.179118,0.106271,0.475000,0.179118
50%,0.500000,0.366071,0.406593,0.500000,0.310096,0.361305,0.149832,0.599026,0.224257,0.149832,0.599026,0.224257
75%,0.618750,0.397826,0.490385,0.584615,0.385417,0.448052,0.193618,0.652174,0.305094,0.193618,0.652174,0.305094
max,1.000000,1.000000,0.700000,1.000000,1.000000,0.700000,0.333333,1.000000,0.430380,0.333333,1.000000,0.430380


In [26]:
df_test_rouge_desc.to_csv('data/statistic/df_test_rouge_desc.csv', index=False)
df_train_rouge_desc.to_csv('data/statistic/df_train_rouge_desc.csv', index=False)
df_validation_rouge_desc.to_csv('data/statistic/df_validation_rouge_desc.csv', index=False)

### BERT

In [27]:
bert_score_df

,P_train,P_train_o,P_test,P_test_o,P_validation,P_validation_o,R_train,R_train_o,R_test,R_test_o,R_validation,R_validation_o,F1_train,F1_train_o,F1_test,F1_test_o,F1_validation,F1_validation_o
0,0.851073,0.830172,0.851073,0.830172,0.862566,0.823095,0.820701,0.841389,0.820701,0.841389,0.844664,0.851107,0.835611,0.835743,0.835611,0.835743,0.853521,0.836866
1,0.837143,0.802147,0.837143,0.802147,0.863262,0.793842,0.812836,0.828479,0.812836,0.828479,0.831809,0.842845,0.824810,0.815100,0.824810,0.815100,0.847244,0.817610
2,0.938886,0.858702,0.938886,0.858702,0.846628,0.793292,0.895425,0.903526,0.895425,0.903526,0.823736,0.819273,0.916641,0.880544,0.916641,0.880544,0.835025,0.806073
3,0.915542,0.780460,0.915542,0.780460,0.901733,0.803790,0.887759,0.828171,0.887759,0.828171,0.840561,0.851488,0.901437,0.803608,0.901437,0.803608,0.870074,0.826952
4,0.820946,0.800748,0.820946,0.800748,0.870939,0.833948,0.808682,0.816048,0.808682,0.816048,0.856189,0.866599,0.814768,0.808326,0.814768,0.808326,0.863501,0.849960
5,0.784017,0.726955,0.784017,0.726955,0.854362,0.794522,0.831832,0.825357,0.831832,0.825357,0.852210,0.831202,0.807217,0.773037,0.807217,0.773037,0.853284,0.812448
6,0.837947,0.815765,0.837947,0.815765,0.956940,0.834840,0.819633,0.854859,0.819633,0.854859,0.883789,0.898623,0.828689,0.834854,0.828689,0.834854,0.918911,0.865558
7,0.940437,0.809727,0.940437,0.809727,0.878161,0.807265,0.811009,0.859867,0.811009,0.859867,0.862651,0.880710,0.870941,0.834044,0.870941,0.834044,0.870336,0.842390
8,1.000000,0.782535,1.000000,0.782535,0.909359,0.782460,1.000000,0.853342,1.000000,0.853342,0.950481,0.886862,1.000000,0.816406,1.000000,0.816406,0.929466,0.831397
9,0.862328,0.844507,0.862328,0.844507,0.899151,0.818273,0.846845,0.856490,0.846845,0.856490,0.858068,0.870399,0.854517,0.850456,0.854517,0.850456,0.878129,0.843531


In [28]:
bert_score_df.to_csv('data/statistic/bert_score.csv', index=False)

In [29]:
bert_score_df.describe()

,P_train,P_train_o,P_test,P_test_o,P_validation,P_validation_o,R_train,R_train_o,R_test,R_test_o,R_validation,R_validation_o,F1_train,F1_train_o,F1_test,F1_test_o,F1_validation,F1_validation_o
count,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000
mean,0.890776,0.810589,0.890776,0.810589,0.882075,0.814422,0.872930,0.859233,0.872930,0.859233,0.856787,0.856922,0.881423,0.833929,0.881423,0.833929,0.869037,0.834965
std,0.068050,0.042245,0.068050,0.042245,0.030344,0.021285,0.066350,0.034324,0.066350,0.034324,0.031837,0.025368,0.064960,0.035881,0.064960,0.035881,0.027564,0.019887
min,0.784017,0.700051,0.784017,0.700051,0.825929,0.782460,0.808682,0.802411,0.808682,0.802411,0.798397,0.818069,0.801404,0.747745,0.801404,0.747745,0.811929,0.806073
25%,0.841228,0.801098,0.841228,0.801098,0.862740,0.794768,0.819900,0.831707,0.819900,0.831707,0.841587,0.842959,0.830419,0.815427,0.830419,0.815427,0.853605,0.816295
50%,0.882311,0.818207,0.882311,0.818207,0.885582,0.811323,0.853082,0.858179,0.853082,0.858179,0.857078,0.851298,0.866465,0.835609,0.866465,0.835609,0.866787,0.834132
75%,0.936222,0.835131,0.936222,0.835131,0.899288,0.832785,0.893508,0.876250,0.893508,0.876250,0.866187,0.878888,0.912840,0.852137,0.912840,0.852137,0.881778,0.848353
max,1.000000,0.861663,1.000000,0.861663,0.956940,0.858153,1.000000,0.940577,1.000000,0.940577,0.950481,0.898623,1.000000,0.885068,1.000000,0.885068,0.929466,0.865719
